In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("311-2014-2015.csv")

/Users/joshua/anaconda/envs/python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (8,17,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
cols = ['Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Status', 'Latitude', 'Longitude']
df = df[cols]

In [36]:
nyc_facilities = pd.read_csv("NYCfac.txt",encoding = 'iso-8859-1')

In [45]:
factype = pd.read_csv("Factype.txt",encoding = 'iso-8859-1')

In [53]:
factype[factype['FT_Map'].str.contains("Fire")]

,FacType,FT_Decode,FT_Report,FT_Map
107,2101,NYC Fire House,NYC Fire House,NYC Fire House


In [ ]:
#Police station = FacType 2001
#Fire House = FacType 2101

In [59]:
pdfd = nyc_facilities[nyc_facilities['FacType'].isin([2001, 2101])]

In [64]:
pdfd_cols = ['Borough','ZipCode', 'FacType','PolicePrct','XCoord', 'YCoord']
pdfd = pdfd[pdfd_cols]

In [82]:
import pyproj

def convert(x, y):
    state_plane = pyproj.Proj(init='ESRI:102718', preserve_units=True)
    wgs = pyproj.Proj(proj='latlong', datum='WGS84', ellps='WGS84')
    lng, lat = pyproj.transform(state_plane, wgs, x, y)
    return lat, lng

In [84]:
pdfd.head()

,Borough,ZipCode,FacType,PolicePrct,XCoord,YCoord
258,BK,11224,2001,60,990869,149279
259,BK,11223,2001,61,995238,155733
260,BK,11214,2001,62,983394,158816
261,BK,11210,2001,63,1000448,168068
262,BK,11204,2001,66,986633,167258


In [105]:
def get_lat(row):
    x = row["XCoord"]
    y = row["YCoord"]
    lat, lon = convert(x, y)
    return lat

def get_lon(row):
    x = row["XCoord"]
    y = row["YCoord"]
    lat, lon = convert(x, y)
    return lon

In [98]:
pdfd["Latitude"] = pdfd.apply(get_lat, axis=1)
pdfd["Longitude"] = pdfd.apply(get_lon, axis=1)

In [109]:
#Police station = FacType 2001
#Fire House = FacType 2101
pdfd['Type'] = pdfd['FacType'].apply(lambda x: "Police" if x == 2001 else "Fire")

In [112]:
pdfd.head()

,Borough,ZipCode,FacType,PolicePrct,XCoord,YCoord,Latitude,Longitude,Type
258,BK,11224,2001,60,990869,149279,40.576410,-73.976173,Police
259,BK,11223,2001,61,995238,155733,40.594120,-73.960435,Police
260,BK,11214,2001,62,983394,158816,40.602589,-74.003083,Police
261,BK,11210,2001,63,1000448,168068,40.627970,-73.941646,Police
262,BK,11204,2001,66,986633,167258,40.625761,-73.991415,Police


In [113]:
pdfd.to_csv("nypd-nyfd-stations.csv", index=False)

In [119]:
df = df[df["Agency"] == "NYPD"]

In [121]:
df.head()

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Status,Latitude,Longitude
0,2016-04-14 02:10:19,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,11436,Open,40.684998,-73.795037
1,2016-04-14 02:10:07,04/14/2016 03:17:04 AM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11221,Closed,40.695371,-73.928720
3,2016-04-14 02:08:10,04/14/2016 03:17:08 AM,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11221,Closed,40.695371,-73.928720
4,2016-04-14 02:07:42,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,10003,Open,40.730038,-73.992632
6,2016-04-14 02:00:05,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11223,Assigned,40.601353,-73.981346


In [130]:
df.Status.value_counts()

Closed        744055
Open            2271
Assigned        1071
Unassigned         7
Draft              2
Name: Status, dtype: int64

In [129]:
df["Closed Date"] = pd.to_datetime(df["Closed Date"], infer_datetime_format=True)

KeyboardInterrupt: 

In [122]:
df["Created Date"] = pd.to_datetime(df["Created Date"])
df["Closed Date"] = pd.to_datetime(df["Closed Date"])

/Users/joshua/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


KeyboardInterrupt: 

In [104]:
df.head()

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Status,Latitude,Longitude
0,04/14/2016 02:10:19 AM,NaN,NYPD,New York City Police Department,Noise - Residential,Loud Talking,Residential Building/House,11436,Open,40.684998,-73.795037
1,04/14/2016 02:10:07 AM,04/14/2016 03:17:04 AM,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11221,Closed,40.695371,-73.928720
2,04/14/2016 02:08:42 AM,NaN,TLC,Taxi and Limousine Commission,For Hire Vehicle Complaint,Car Service Company Complaint,Street,11372,Open,40.746943,-73.890500
3,04/14/2016 02:08:10 AM,04/14/2016 03:17:08 AM,NYPD,New York City Police Department,Animal Abuse,Neglected,Residential Building/House,11221,Closed,40.695371,-73.928720
4,04/14/2016 02:07:42 AM,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,10003,Open,40.730038,-73.992632
